In [1]:
import findspark
findspark.init()

# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my app").master("local").getOrCreate()

# get context from the session
sc = spark.sparkContext

# Data loading & schema configuration

Field types
* EXT1 ~ OPN10: float
* country: string 

In [80]:
cols = ['EXT1', 'EXT2', 'EXT3', 'EXT4', 'EXT5', 'EXT6', 'EXT7', 'EXT8', 'EXT9', 'EXT10', 'EST1', 'EST2', 'EST3', 'EST4', 'EST5', 'EST6', 'EST7', 'EST8', 'EST9', 'EST10', 'AGR1', 'AGR2', 'AGR3', 'AGR4', 'AGR5', 'AGR6', 'AGR7', 'AGR8', 'AGR9', 'AGR10', 'CSN1', 'CSN2', 'CSN3', 'CSN4', 'CSN5', 'CSN6', 'CSN7', 'CSN8', 'CSN9', 'CSN10', 'OPN1', 'OPN2', 'OPN3', 'OPN4', 'OPN5', 'OPN6', 'OPN7', 'OPN8', 'OPN9', 'OPN10', 'EXT1_E', 'EXT2_E', 'EXT3_E', 'EXT4_E', 'EXT5_E', 'EXT6_E', 'EXT7_E', 'EXT8_E', 'EXT9_E', 'EXT10_E', 'EST1_E', 'EST2_E', 'EST3_E', 'EST4_E', 'EST5_E', 'EST6_E', 'EST7_E', 'EST8_E', 'EST9_E', 'EST10_E', 'AGR1_E', 'AGR2_E', 'AGR3_E', 'AGR4_E', 'AGR5_E', 'AGR6_E', 'AGR7_E', 'AGR8_E', 'AGR9_E', 'AGR10_E', 'CSN1_E', 'CSN2_E', 'CSN3_E', 'CSN4_E', 'CSN5_E', 'CSN6_E', 'CSN7_E', 'CSN8_E', 'CSN9_E', 'CSN10_E', 'OPN1_E', 'OPN2_E', 'OPN3_E', 'OPN4_E', 'OPN5_E', 'OPN6_E', 'OPN7_E', 'OPN8_E', 'OPN9_E', 'OPN10_E', 'dateload', 'screenw', 'screenh', 'introelapse', 'testelapse', 'endelapse', 'IPC', 'country', 'lat_appx_lots_of_err', 'long_appx_lots_of_err']

In [38]:
from pyspark.sql.types import *

schema = StructType([ StructField(col, StringType(), False) if col == 'country' else StructField(col, FloatType(), False) for col in cols ]  )
raw_data = spark.read.format("csv").option("header","true").option("delimiter", "\t").schema(schema).load("data-final.csv")

In [39]:
cols_to_use = cols[:50] + ['country']
cols_to_drop = [ col for col in cols if col not in cols_to_use ]
raw_data = raw_data.drop(*cols_to_drop)

In [40]:
raw_data.printSchema()

root
 |-- EXT1: float (nullable = true)
 |-- EXT2: float (nullable = true)
 |-- EXT3: float (nullable = true)
 |-- EXT4: float (nullable = true)
 |-- EXT5: float (nullable = true)
 |-- EXT6: float (nullable = true)
 |-- EXT7: float (nullable = true)
 |-- EXT8: float (nullable = true)
 |-- EXT9: float (nullable = true)
 |-- EXT10: float (nullable = true)
 |-- EST1: float (nullable = true)
 |-- EST2: float (nullable = true)
 |-- EST3: float (nullable = true)
 |-- EST4: float (nullable = true)
 |-- EST5: float (nullable = true)
 |-- EST6: float (nullable = true)
 |-- EST7: float (nullable = true)
 |-- EST8: float (nullable = true)
 |-- EST9: float (nullable = true)
 |-- EST10: float (nullable = true)
 |-- AGR1: float (nullable = true)
 |-- AGR2: float (nullable = true)
 |-- AGR3: float (nullable = true)
 |-- AGR4: float (nullable = true)
 |-- AGR5: float (nullable = true)
 |-- AGR6: float (nullable = true)
 |-- AGR7: float (nullable = true)
 |-- AGR8: float (nullable = true)
 |-- AGR9: fl

In [41]:
raw_data.show(1)

+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+-------+
|EXT1|EXT2|EXT3|EXT4|EXT5|EXT6|EXT7|EXT8|EXT9|EXT10|EST1|EST2|EST3|EST4|EST5|EST6|EST7|EST8|EST9|EST10|AGR1|AGR2|AGR3|AGR4|AGR5|AGR6|AGR7|AGR8|AGR9|AGR10|CSN1|CSN2|CSN3|CSN4|CSN5|CSN6|CSN7|CSN8|CSN9|CSN10|OPN1|OPN2|OPN3|OPN4|OPN5|OPN6|OPN7|OPN8|OPN9|OPN10|country|
+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+-------+
| 4.0| 1.0| 5.0| 2.0| 5.0| 1.0| 5.0| 2.0| 4.0|  1.0| 1.0| 4.0| 4.0| 2.0| 2.0| 2.0| 2.0| 2.0| 3.0|  2.0| 2.0| 5.0| 2.0| 4.0| 2.0| 3.0| 2.0| 4.0| 3.0|  4.0| 3.0| 4.0| 3.0| 2.0| 2.0| 4.0| 4.0| 2.0| 4.0|  4.0|

In [43]:
raw_data = raw_data.filter((raw_data['country'] == 'KR') | (raw_data['country'] == 'US'))

In [45]:
raw_data.show(10)

+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+-------+
|EXT1|EXT2|EXT3|EXT4|EXT5|EXT6|EXT7|EXT8|EXT9|EXT10|EST1|EST2|EST3|EST4|EST5|EST6|EST7|EST8|EST9|EST10|AGR1|AGR2|AGR3|AGR4|AGR5|AGR6|AGR7|AGR8|AGR9|AGR10|CSN1|CSN2|CSN3|CSN4|CSN5|CSN6|CSN7|CSN8|CSN9|CSN10|OPN1|OPN2|OPN3|OPN4|OPN5|OPN6|OPN7|OPN8|OPN9|OPN10|country|
+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+-------+
| 4.0| 3.0| 4.0| 3.0| 3.0| 3.0| 5.0| 3.0| 4.0|  3.0| 2.0| 4.0| 4.0| 2.0| 4.0| 2.0| 2.0| 2.0| 4.0|  4.0| 1.0| 2.0| 1.0| 5.0| 3.0| 5.0| 3.0| 4.0| 4.0|  5.0| 3.0| 2.0| 4.0| 2.0| 1.0| 4.0| 4.0| 2.0| 2.0|  5.0|

# Data processing steps

1. Imputer: replace <code>None</code> with average values
1. VectorAssembler: create a feature vector with all scores for the 50 questions
1. PCA: reduce the dimensionality into 10
1. GaussianMixture (or KMeans): clustering
1. StringIndexer: represent country attributes with integer IDs
1. OneHotEncoder: interpret country IDs with one hot vectors
1. VectorAssembler: concatenate cluster probability vector and one-hot encoded country ID vector
1. Correlation: compute correlation matrix

In [48]:
from pyspark.ml.feature import Imputer

imputer=Imputer(inputCols=raw_data.columns[:50],outputCols=raw_data.columns[:50])
raw_data = imputer.fit(raw_data).transform(raw_data)

In [55]:
from pyspark.ml.feature import VectorAssembler

input_cols = raw_data.columns[:50]
assembler = VectorAssembler(inputCols=input_cols,outputCol="raw_features")
df1=assembler.transform(raw_data)

In [56]:
from pyspark.ml.feature import PCA

pca = PCA(k=10, inputCol="raw_features", outputCol="features")
df2 = pca.fit(df1).transform(df1)

In [58]:
from pyspark.ml.clustering import GaussianMixture
gm = GaussianMixture(featuresCol="features", k=2, tol=0.0001, seed=10)
df3 = gm.fit(df2).transform(df2)

In [60]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="country", outputCol="country_int")
df4 = indexer.fit(df3).transform(df3)

In [75]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCol="country_int", outputCol="country_onehot", dropLast=False)
df5 = encoder.transform(df4)

In [77]:
assembler = VectorAssembler(inputCols=["probability", "country_onehot"],outputCol="corr_features")
df6=assembler.transform(df5)

In [78]:
df6.select("corr_features").show(5, truncate=False)

+--------------------------------------------------+
|corr_features                                     |
+--------------------------------------------------+
|[0.0024932054464815914,0.9975067945535183,1.0,0.0]|
|[0.3921775721861428,0.6078224278138572,1.0,0.0]   |
|[0.03323653703287065,0.9667634629671294,1.0,0.0]  |
|[0.9357516669480505,0.06424833305194967,1.0,0.0]  |
|[2.0509958008700714E-4,0.9997949004199129,1.0,0.0]|
+--------------------------------------------------+
only showing top 5 rows



In [79]:
from pyspark.ml.stat import Correlation
Correlation.corr(df6, 'corr_features', 'pearson').first()

Row(pearson(corr_features)=DenseMatrix(4, 4, [1.0, -1.0, -0.0112, 0.0112, -1.0, 1.0, 0.0112, -0.0112, -0.0112, 0.0112, 1.0, -1.0, 0.0112, -0.0112, -1.0, 1.0], False))